In [ ]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import torch
from torch import nn
from torch import optim
import os
import numpy as np


Mounted at /content/drive


In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import TensorDataset, DataLoader, Subset

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on cuda
Flower 1.15.2 / PyTorch 2.5.1+cu124


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def carregar_dados_no_padding(train_data, test_data, test_size=0.2, random_state=42):
    # Load data from CSV file
    scaler = StandardScaler()

    train_data = pd.read_csv(train_data)
    val_data = pd.read_csv(test_data)

    train_data_filtered = train_data.loc[:, ~(train_data == 0).any(axis=0)]
    val_data_filtered = val_data.loc[:, ~(val_data == 0).any(axis=0)]


    # Split data into features (X) and targets (y) ("hysteresis")
    X = train_data_filtered.drop(['hysteresis', 'joule'], axis=1)
    y = train_data_filtered[['hysteresis', 'joule']]

    # Perform train-test split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=random_state)

    X_test = val_data_filtered.drop(['hysteresis', 'joule'], axis=1)
    y_test = val_data_filtered[['hysteresis', 'joule']]

    X_train = scaler.fit_transform(X_train.values)
    X_val = scaler.transform(X_val.values)
    X_test = scaler.transform(X_test.values)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)


    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor

In [ ]:
import matplotlib.pyplot as plt
import torch

def plot_comparacao(y_true, y_pred):
    # Converta para numpy se estiverem em tensores
    if torch.is_tensor(y_true):
        y_true = y_true.cpu().numpy()
    if torch.is_tensor(y_pred):
        y_pred = y_pred.cpu().numpy()

    # Calcular métricas de avaliação
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    # Plotar a comparação
    plt.figure(figsize=(10, 8))
    plt.scatter(y_true, y_pred, color='blue', alpha=0.5, s=50, label='Previsões')

    # Linha de referência (Y = X)
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', linewidth=2, label='Linha de Referência')

    # Adicionar título e legendas
    plt.xlabel('Valores Reais', fontsize=12)
    plt.ylabel('Valores Previstos', fontsize=12)
    plt.title('Comparação entre Valores Reais e Previstos', fontsize=14)
    plt.legend()

    # Exibir métricas no gráfico
    plt.text(min_val, max_val * 0.95, f'MAE: {mae:.4f}', fontsize=12, color='green')
    plt.text(min_val, max_val * 0.90, f'MSE: {mse:.4f}', fontsize=12, color='green')
    plt.text(min_val, max_val * 0.85, f'RMSE: {rmse:.4f}', fontsize=12, color='green')

    # Adicionar uma grade
    plt.grid(True, linestyle='--', alpha=0.7)

    # Exibir o gráfico
    plt.show()


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim),
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, output_dim, num_layers, hidden_dim):
        super(Net, self).__init__()
        layers = []

        # Construindo as camadas com base no número de camadas
        for i in range(num_layers):
            layers.append(nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim))
            layers.append(nn.ReLU())

        # Guardar o índice da penúltima camada
        self.penultimate_layer_index = len(layers) - 2

        # Adicionar a camada final de saída
        layers.append(nn.Linear(hidden_dim, output_dim))

        # Definir a sequência de camadas como parte da rede
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        penultimate_output = None

        # Executar o forward manualmente para capturar a penúltima camada
        for i, layer in enumerate(self.network):
            x = layer(x)
            if i == self.penultimate_layer_index:
                penultimate_output = x  # Armazenar a saída da penúltima camada

        # Retornar a saída final e a penúltima camada
        return x, penultimate_output

In [ ]:
from torch import tensor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.optim.lr_scheduler import ReduceLROnPlateau


def train(model,  epochs, optimizer, trainloader, valloader):

    criterion = nn.MSELoss()
    history = {"train_loss": [], "val_loss": []}

    # Coloca o modelo no dispositivo especificado
    model.to(device)

    for epoch in range(epochs):
        # Etapa de treinamento
        model.train()
        train_loss = 0.0

        for inputs, targets in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs, _ = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)  # Acumula a perda ponderada pelo batch size

        train_loss /= len(trainloader.dataset)  # Média da perda por amostra
        history["train_loss"].append(train_loss)

        # Etapa de validação
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in valloader:
                inputs, targets = inputs.to(device), targets.to(device)

                outputs, _ = model(inputs)
                loss = criterion(outputs, targets)

                val_loss += loss.item() * inputs.size(0)

        val_loss /= len(valloader.dataset)  # Média da perda por amostra
        history["val_loss"].append(val_loss)

        #print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.8f}, Val Loss: {val_loss:.8f}")

def test(net, testloader):
    """Evaluate the network on the entire test set for a regression problem."""
    criterion = torch.nn.MSELoss()  # Use MSE as the loss
    total_loss, total_mse = 0.0, 0.0
    net.eval()

    with torch.no_grad():
        for batch in testloader:
            images, targets = batch[0].to(DEVICE), batch[1].to(DEVICE)
            outputs , _ = net(images)

            # Calculate the loss (MSE)
            loss = criterion(outputs, targets)
            total_loss += loss.item()

            # Calculate the MSE for accuracy
            mse = torch.mean((outputs - targets) ** 2)
            total_mse += mse.item()

    avg_loss = total_loss / len(testloader.dataset)
    avg_mse = total_mse / len(testloader)  # Average per batch for mse

    return avg_loss, avg_mse

In [ ]:
def change_dataset(folder_path):
    folder_path = folder_path
    os.chdir(folder_path)

    X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor = carregar_dados_no_padding('dados_de_treino.csv','dados_de_teste.csv')

    X_train_tensor = X_train_tensor.to(device)
    y_train_tensor = y_train_tensor.to(device)
    X_val_tensor = X_val_tensor.to(device)
    y_val_tensor = y_val_tensor.to(device)
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)

    histeresis_train_tensor = [pair[0].item() for pair in y_train_tensor]
    histeresis_train_tensor = torch.tensor(histeresis_train_tensor, dtype=torch.float32).to(device)
    histeresis_train_tensor = histeresis_train_tensor.unsqueeze(1)

    joule_train_tensor = [pair[1].item() for pair in y_train_tensor]
    joule_train_tensor = torch.tensor(joule_train_tensor, dtype=torch.float32).to(device)
    joule_train_tensor = joule_train_tensor.unsqueeze(1)

    histeresis_val_tensor = [pair[0].item() for pair in y_val_tensor]
    histeresis_val_tensor = torch.tensor(histeresis_val_tensor, dtype=torch.float32).to(device)
    histeresis_val_tensor = histeresis_val_tensor.unsqueeze(1)

    joule_val_tensor = [pair[1].item() for pair in y_val_tensor]
    joule_val_tensor = torch.tensor(joule_val_tensor, dtype=torch.float32).to(device)
    joule_val_tensor = joule_val_tensor.unsqueeze(1)

    histeresis_test_tensor = [pair[0].item() for pair in y_test_tensor]
    histeresis_test_tensor = torch.tensor(histeresis_test_tensor, dtype=torch.float32).to(device)
    histeresis_test_tensor = histeresis_test_tensor.unsqueeze(1)

    joule_test_tensor = [pair[1].item() for pair in y_test_tensor]
    joule_test_tensor = torch.tensor(joule_test_tensor, dtype=torch.float32).to(device)
    joule_test_tensor = joule_test_tensor.unsqueeze(1)

    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor, histeresis_train_tensor, joule_train_tensor, histeresis_val_tensor, joule_val_tensor, histeresis_test_tensor, joule_test_tensor

In [ ]:
X_train_tensorn, y_train_tensorn, X_val_tensorn, y_val_tensorn, X_test_tensorn, y_test_tensorn, histeresis_train_tensorn, joule_train_tensorn, histeresis_val_tensorn, joule_val_tensorn, histeresis_test_tensorn, joule_test_tensorn = change_dataset('/content/drive/MyDrive/FederatedLearning-main/Data/IPMSM_datasets/dataset_for_iron_losses_of_IPMSMs/Nabla')
X_train_tensor2d, y_train_tensor2d, X_val_tensor2d, y_val_tensor2d, X_test_tensor2d, y_test_tensor2d, histeresis_train_tensor2d, joule_train_tensor2d, histeresis_val_tensor2d, joule_val_tensor2d, histeresis_test_tensor2d, joule_test_tensor2d = change_dataset('/content/drive/MyDrive/FederatedLearning-main/Data/IPMSM_datasets/dataset_for_iron_losses_of_IPMSMs/2D')
X_train_tensorv, y_train_tensorv, X_val_tensorv, y_val_tensorv, X_test_tensorv, y_test_tensorv, histeresis_train_tensorv, joule_train_tensorv, histeresis_val_tensorv, joule_val_tensorv, histeresis_test_tensorv, joule_test_tensorv = change_dataset('/content/drive/MyDrive/FederatedLearning-main/Data/IPMSM_datasets/dataset_for_iron_losses_of_IPMSMs/V')


In [ ]:
#datasets para treinamento do autoencoder
NUM_CLIENTS = 10
BATCH_SIZE = 512

auto_train_datasetv = TensorDataset(X_train_tensorv, X_train_tensorv)
auto_val_datasetv = TensorDataset(X_val_tensorv, X_val_tensorv)
auto_test_datasetv = TensorDataset(X_test_tensorv, X_test_tensorv)

auto_train_datasetn = TensorDataset(X_train_tensorn, X_train_tensorn)
auto_val_datasetn = TensorDataset(X_val_tensorn, X_val_tensorn)
auto_test_datasetn = TensorDataset(X_test_tensorn, X_test_tensorn)

auto_train_dataset2d = TensorDataset(X_train_tensor2d, X_train_tensor2d)
auto_val_dataset2d = TensorDataset(X_val_tensor2d,X_val_tensor2d)
auto_test_dataset2d = TensorDataset(X_test_tensor2d, X_test_tensor2d)

In [ ]:
batch_size = 256
auto_train_loader_v = DataLoader(auto_train_datasetv, batch_size=batch_size, shuffle=True)
auto_val_loader_v = DataLoader(auto_val_datasetv, batch_size=batch_size, shuffle=False)
auto_test_loader_v = DataLoader(auto_test_datasetv, batch_size=batch_size, shuffle=False)

auto_train_loader_n = DataLoader(auto_train_datasetn, batch_size=batch_size, shuffle=True)
auto_val_loader_n = DataLoader(auto_val_datasetn, batch_size=batch_size, shuffle=False)
auto_test_loader_n = DataLoader(auto_test_datasetn, batch_size=batch_size, shuffle=False)

auto_train_loader_2d = DataLoader(auto_train_dataset2d, batch_size=batch_size, shuffle=True)
auto_val_loader_2d = DataLoader(auto_val_dataset2d, batch_size=batch_size, shuffle=False)
auto_test_loader_2d = DataLoader(auto_test_dataset2d, batch_size=batch_size, shuffle=False)

In [ ]:
modeln = Autoencoder(X_train_tensorn.shape[1],20).to(device)
optimizern = optim.Adam(modeln.parameters(), lr=0.001)

model2d = Autoencoder(X_train_tensor2d.shape[1],20).to(device)
optimizer2d = optim.Adam(model2d.parameters(), lr=0.001)

modelv = Autoencoder(X_train_tensorv.shape[1],20).to(device)
optimizerv = optim.Adam(modelv.parameters(), lr=0.001)

In [ ]:
train(modeln,100,optimizern,auto_train_loader_n,auto_val_loader_n)

In [ ]:
train(model2d,100,optimizer2d,auto_train_loader_2d,auto_val_loader_2d)

In [ ]:
train(modelv,100,optimizerv,auto_train_loader_v,auto_val_loader_v)

In [ ]:
test(modeln,auto_test_loader_n)

(4.3652657468115796e-08, 1.0589215698449775e-05)

In [ ]:
test(model2d,auto_test_loader_2d)

(3.27937542696051e-07, 8.338243519813802e-05)

In [ ]:
test(modelv,auto_test_loader_v)

(3.481198886521607e-07, 8.713990475945665e-05)

In [ ]:
modeln.eval()
modelv.eval()
model2d.eval()

encoded_dataset_n = []
encoded_dataset_v = []
encoded_dataset_2d = []

vencoded_dataset_n = []
vencoded_dataset_v = []
vencoded_dataset_2d = []

tencoded_dataset_n = []
tencoded_dataset_v = []
tencoded_dataset_2d = []

with torch.no_grad():
    for x in X_train_tensorn:
        x = x.unsqueeze(0).to(device)
        latent = modeln.encoder(x)
        encoded_dataset_n.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_train_tensor2d:
        x = x.unsqueeze(0).to(device)
        latent = model2d.encoder(x)
        encoded_dataset_2d.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_train_tensorv:
        x = x.unsqueeze(0).to(device)
        latent = modelv.encoder(x)
        encoded_dataset_v.append(latent.squeeze(0).cpu())

encoded_dataset_n = torch.stack(encoded_dataset_n)
encoded_dataset_2d = torch.stack(encoded_dataset_2d)
encoded_dataset_v = torch.stack(encoded_dataset_v)

print(f"Shape do dataset codificado: {encoded_dataset_n.shape}")
print(f"Shape do dataset codificado: {encoded_dataset_2d.shape}")
print(f"Shape do dataset codificado: {encoded_dataset_v.shape}")

with torch.no_grad():
    for x in X_val_tensorn:
        x = x.unsqueeze(0).to(device)
        latent = modeln.encoder(x)
        vencoded_dataset_n.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_val_tensor2d:
        x = x.unsqueeze(0).to(device)
        latent = model2d.encoder(x)
        vencoded_dataset_2d.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_val_tensorv:
        x = x.unsqueeze(0).to(device)
        latent = modelv.encoder(x)
        vencoded_dataset_v.append(latent.squeeze(0).cpu())

vencoded_dataset_n = torch.stack(vencoded_dataset_n)
vencoded_dataset_2d = torch.stack(vencoded_dataset_2d)
vencoded_dataset_v = torch.stack(vencoded_dataset_v)

print(f"Shape do dataset codificado: {vencoded_dataset_n.shape}")
print(f"Shape do dataset codificado: {vencoded_dataset_2d.shape}")
print(f"Shape do dataset codificado: {vencoded_dataset_v.shape}")

with torch.no_grad():
    for x in X_test_tensorn:
        x = x.unsqueeze(0).to(device)
        latent = modeln.encoder(x)
        tencoded_dataset_n.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_test_tensor2d:
        x = x.unsqueeze(0).to(device)
        latent = model2d.encoder(x)
        tencoded_dataset_2d.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_test_tensorv:
        x = x.unsqueeze(0).to(device)
        latent = modelv.encoder(x)
        tencoded_dataset_v.append(latent.squeeze(0).cpu())

tencoded_dataset_n = torch.stack(tencoded_dataset_n)
tencoded_dataset_2d = torch.stack(tencoded_dataset_2d)
tencoded_dataset_v = torch.stack(tencoded_dataset_v)

print(f"Shape do dataset codificado: {tencoded_dataset_n.shape}")
print(f"Shape do dataset codificado: {tencoded_dataset_2d.shape}")
print(f"Shape do dataset codificado: {tencoded_dataset_v.shape}")


Shape do dataset codificado: torch.Size([18436, 20])
Shape do dataset codificado: torch.Size([19328, 20])
Shape do dataset codificado: torch.Size([19026, 20])
Shape do dataset codificado: torch.Size([4610, 20])
Shape do dataset codificado: torch.Size([4832, 20])
Shape do dataset codificado: torch.Size([4757, 20])
Shape do dataset codificado: torch.Size([4609, 20])
Shape do dataset codificado: torch.Size([4831, 20])
Shape do dataset codificado: torch.Size([4756, 20])


In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 512

train_datasetv = TensorDataset(encoded_dataset_v, y_train_tensorv)
val_datasetv = TensorDataset(vencoded_dataset_v, y_val_tensorv)
test_datasetv = TensorDataset(tencoded_dataset_v, y_test_tensorv)

train_datasetn = TensorDataset(encoded_dataset_n, y_train_tensorn)
val_datasetn = TensorDataset(vencoded_dataset_n, y_val_tensorn)
test_datasetn = TensorDataset(tencoded_dataset_n, y_test_tensorn)


train_dataset2d = TensorDataset(encoded_dataset_2d, joule_train_tensor2d)
val_dataset2d = TensorDataset(vencoded_dataset_2d, joule_val_tensor2d)
test_dataset2d = TensorDataset(tencoded_dataset_2d, joule_test_tensor2d)

def split_tensor_dataset(tensor_data):
    subset_size = len(tensor_data) // NUM_CLIENTS
    indices = np.random.permutation(len(tensor_data))
    subsets = []
    for i in range(NUM_CLIENTS):
        start = i * subset_size
        end = (i + 1) * subset_size if i != NUM_CLIENTS - 1 else len(tensor_data)
        subset_indices = indices[start:end]
        subsets.append(Subset(tensor_data, subset_indices))
    return subsets

    # Divide o dataset entre os clientes
train_subsetsn = split_tensor_dataset(train_datasetn)
val_subsetsn = split_tensor_dataset(val_datasetn)

train_subsetsv = split_tensor_dataset(train_datasetv)
val_subsetsv = split_tensor_dataset(val_datasetv)

train_subsets2d = split_tensor_dataset(train_dataset2d)
val_subsets2d = split_tensor_dataset(val_dataset2d)

def load_datasets(partition_id: int):
    # Converte tensores para TensorDatasets

    if partition_id % 3 == 0:
        train_subsets = train_subsetsv
        val_subsets = val_subsetsv
        test_dataset = test_datasetv
    elif partition_id % 2 == 0:
        train_subsets = train_subsetsn
        val_subsets = val_subsetsn
        test_dataset = test_datasetn
    else:
        train_subsets = train_subsets2d
        val_subsets = val_subsets2d
        test_dataset = test_dataset2d

    # Retorna o DataLoader específico para o cliente com base no partition_id
    train_loader = DataLoader(train_subsets[partition_id], batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subsets[partition_id], batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, val_loader, test_loader

In [ ]:
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)


    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, 100, optim.Adam(self.net.parameters()),self.trainloader,self.valloader)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [ ]:
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net(20, 2, 16, 40).to(DEVICE)

    # Get data partition ID from node_config
    partition_id = context.node_config["partition-id"]

    # Ensure the dataset loading function supports partition_id, or adjust it accordingly
    try:
        trainloader, valloader, _ = load_datasets(partition_id=partition_id)
    except TypeError:
        print(f"load_datasets() does not support partition_id, modifying call.")
        trainloader, valloader, _ = load_datasets()  # Adjust to match your actual function signature

    # Create a Flower client representing the organization
    return FlowerClient(net, trainloader, valloader).to_client()

# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [ ]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [ ]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [ ]:
from typing import Dict, Tuple, Optional
from sklearn.metrics import mean_squared_error, mean_absolute_error
def evaluate(server_round: int, parameters: flwr.common.NDArrays, config: Dict[str, float]) -> Optional[Tuple[float, Dict[str, float]]]:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Criar o modelo e transferi-lo para o dispositivo
    net = Net(20, 2, 16, 40).to(DEVICE)

    # Carregar o DataLoader para os dados de teste
    _, _, testloader = load_datasets(0)  # Ajuste conforme seu código para carregar dados

    # Atualizar o modelo com os parâmetros mais recentes
    set_parameters(net, parameters)
    def test(net, testloader):
        """Evaluate the network on the entire test set for a regression problem."""
        mse_criterion = torch.nn.MSELoss()  # Mean Squared Error
        mae_criterion = torch.nn.L1Loss()  # Mean Absolute Error
        total_mse, total_mae, total_mape = 0.0, 0.0, 0.0
        net.eval()

        with torch.no_grad():
            for batch in testloader:
                images, targets = batch[0].to(DEVICE), batch[1].to(DEVICE)
                outputs, _ = net(images)

                # Calculate MSE
                mse = mse_criterion(outputs, targets)
                total_mse += mse.item()

                # Calculate MAE
                mae = mae_criterion(outputs, targets)
                total_mae += mae.item()

                # Calculate MAPE (avoid division by zero)
                epsilon = 1e-8  # Small value to avoid division by zero
                mape = torch.mean(torch.abs((outputs - targets) / (targets + epsilon))) * 100
                total_mape += mape.item()

        # Average metrics per batch
        avg_mse = total_mse / len(testloader)
        avg_mae = total_mae / len(testloader)
        avg_mape = total_mape / len(testloader)
        return avg_mse,avg_mae,avg_mape

    avg_mse,avg_mae,avg_mape = test(net,testloader)
    return avg_mse, {"MAE": avg_mae, "MAPE": avg_mape}  # Retornando MSE e um dicionário com MAE e MAPE

In [ ]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """

    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
        evaluate_fn=evaluate  # Função de avaliação no servidor
    )

    # Configure the server for 5 rounds of training
    config = ServerConfig(num_rounds=15)

    return ServerAppComponents(strategy=strategy, config=config)


# Create a new server instance with the updated FedAvg strategy
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=15, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=2279) 2025-03-07 17:33:42.421290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=2279) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=2279) E0000 00:00:1741368822.444369    2279 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=2279) E0000 00:00:1741368822.450750    2279 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO :      Received initial parameters from one random client
INFO :      Starting

Centralizado:
Nabla joule model:

Test MSE: 0.007116461172699928

Test MAE: 0.048829417675733566

Test MAPE: 31.627193093299866%

federado:

MSE: 0.0093, MAE: 0.0513, MAPE: 33.53%

Centralizado: Nabla hysteresis model:

Test MSE: 0.004404906183481216

Test MAE: 0.04279777407646179

Test MAPE: 90.69336652755737%

federado:

MSE: 0.0060, MAE: 0.0514, MAPE: 77.05%

Centralizado: V hysteresis model:

Test MSE: 0.0009960791794583201

Test MAE: 0.021568451076745987

Test MAPE: 13.8874351978302%

